In [2]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import requests
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [25]:
train_pos = pd.read_csv("Data/IMDb/train/imdb_train_pos.txt", sep="\n", header=None, names=['review'])
train_pos['positive']=1
train_pos.head()

,review,positive
0,"For fans of Chris Farley, this is probably his...",1
1,"Fantastic, Madonna at her finest, the film is ...",1
2,From a perspective that it is possible to make...,1
3,What is often neglected about Harold Lloyd is ...,1
4,You'll either love or hate movies such as this...,1


In [27]:
train_neg = pd.read_csv("Data/IMDb/train/imdb_train_neg.txt", sep="\n", header=None, names=['review'])
train_neg['positive']=0
train_neg.head()

,review,positive
0,"A terrible deception: controversial film, winn...",0
1,Well I guess I know the answer to that questio...,0
2,I really liked the movie 'The Emporer's New Gr...,0
3,Thats what this movie really takes. A big piec...,0
4,I was looking for a documentary of the same jo...,0


In [29]:
print(train_pos.shape)
print(train_neg.shape)

(7483, 2)
(7517, 2)
